In [43]:
import os
from openai import OpenAI
import markdown
import base64

In [45]:
# 读取api key的txt文件
with open(r"D:\Desktop\其他事务\gpt_api_key.txt", 'r') as f:
    API_KEY = f.read() 

In [46]:
model_name = "gpt-4o-mini-2024-07-18"
# model_name = "gpt-4o-2024-08-06"
if model_name == "gpt-4o-mini-2024-07-18":
    model_abbr = "mini"
else:
    model_abbr = "4o"
client = OpenAI(
    api_key=API_KEY
)

In [47]:
# 读取jsonl文件
import json
FILE_PATH = r"D:\Desktop\intelligent_customer_service\q_man.jsonl"
# 读取jsonl文件的第一个元素
with open(FILE_PATH, 'r', encoding='utf-8') as file:
    first_line = file.readline().strip()
    first_element = json.loads(first_line)
print("jsonl文件的第一个元素：", first_element)

jsonl文件的第一个元素： {'产品信息查询_规格大小': '这个电视机是多大的呢'}


In [48]:
# 将jsonl文件每一行元素的值转换为列表，e.g.
# 输入：{"q_aspect": "价格", "q_content": "价格贵吗？"}
# 输出：{"q_aspect": "价格", "q_content": ["价格贵吗？"]}
import json
# 读取jsonl文件并转换内容
def convert_jsonl_values_to_list(file_path):
    converted_lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            item = json.loads(line.strip())
            # 将每个字典的值转换为列表
            for key, value in item.items():
                item[key] = [value]
            converted_lines.append(item)
    return converted_lines
# 使用函数
converted_lines = convert_jsonl_values_to_list(FILE_PATH)
# 打印转换后的前几个元素以验证
print("转换后的数据(前3个元素):")
for item in converted_lines[:3]:
    print(item)
# 如果需要,可以将转换后的数据写回文件
# with open('converted_file.jsonl', 'w', encoding='utf-8') as outfile:
#     for item in converted_lines:
#         json.dump(item, outfile, ensure_ascii=False)
#         outfile.write('\n')

转换后的数据(前3个元素):
{'产品信息查询_规格大小': ['这个电视机是多大的呢']}
{'产品信息查询_价格': ['这个价格怎么比我上次买的便宜那么多？']}
{'产品信息查询_库存': ['紫色款41码的，还有货吗']}


In [49]:
total_num_gen = 1500
count = 0
begin, end = 2, 12
line_count = len(converted_lines)
per_aspect_num_gen = total_num_gen/line_count
for line in converted_lines[begin:end]:
    q_aspect = next(iter(line.keys())) #只有一个key
    exp_q = list(line.values())
    broad, specific = q_aspect.split('_')
    # system_prompt = f""
    user_prompt = f'''请模拟电商购物的不同顾客，提出关于{specific}的{broad}方面的{per_aspect_num_gen}个问题。
    例如：{exp_q[0]}
    '''
    completion = client.chat.completions.create(
        model = model_name,
        messages=[
            # {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        frequency_penalty=0,
        presence_penalty= -0.5,
        max_tokens=4096,
        temperature=0,
        top_p=0.2,
        n=1,
        seed=42
        )
    reply = completion.choices[0].message.content
    # 将reply append到list变量line[q_aspect]中
    line[q_aspect].append(reply)

In [51]:
OUTPUT_FILE_PATH = \
    fr"D:\Desktop\intelligent_customer_service\\{model_abbr}_reply_{begin}_{end}.jsonl" 
# 将list变量reply转换为jsonl文件
with open(OUTPUT_FILE_PATH, 'w', encoding='utf-8') as outfile:
    for item in converted_lines[begin:end]:
        json.dump(item, outfile, ensure_ascii=False)
        outfile.write('\n')